In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split # split  data into training and testing sets
from sklearn.metrics import plot_confusion_matrix # draws a confusion matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
import xgboost as xgb
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import time
import pickle

In [2]:
root = "../../../../../"

In [3]:
df = pd.read_csv(root + "datasets/multiclass/processed/CICDDoS_corr.csv", index_col=[0])

In [4]:
df.head()

Total Fwd Packets   Total Backward Packets   Total Length of Bwd Packets  \
0                 2.0                      0.0                           0.0   
1                 2.0                      0.0                           0.0   
2                 2.0                      0.0                           0.0   
3                 2.0                      0.0                           0.0   
4                 2.0                      0.0                           0.0   

    Fwd Packet Length Min   Fwd Packet Length Std  Bwd Packet Length Max  \
0                  1368.0                     0.0                    0.0   
1                  1325.0                     0.0                    0.0   
2                  1472.0                     0.0                    0.0   
3                  1472.0                     0.0                    0.0   
4                  1472.0                     0.0                    0.0   

    Bwd Packet Length Min   Bwd Packet Length Mean  Flow Bytes/s  \
0                     0.0                      0.0  2.736000e+09   
1                     0.0                      0.0  2.650000e+09   
2                     0.0                      0.0  1.472000e+09   
3                     0.0                      0.0  6.400000e+07   
4                     0.0                      0.0  2.944000e+09   

    Flow Packets/s  ...   Subflow Bwd Packets   Subflow Bwd Bytes  \
0       2000000.00  ...                   0.0                 0.0   
1       2000000.00  ...                   0.0                 0.0   
2       1000000.00  ...                   0.0                 0.0   
3         43478.26  ...                   0.0                 0.0   
4       2000000.00  ...                   0.0                 0.0   

   Init_Win_bytes_forward   Init_Win_bytes_backward   min_seg_size_forward  \
0                     1.0                       1.0                    1.0   
1                     1.0                       1.0                    0.0   
2                     1.0                       1.0                   14.0   
3                     1.0                       1.0                    1.0   
4                     1.0                       1.0                    1.0   

   Active Mean   Active Std   Idle Std   Idle Min      Label  
0          0.0          0.0        0.0        0.0  DrDoS_DNS  
1          0.0          0.0        0.0        0.0  DrDoS_DNS  
2          0.0          0.0        0.0        0.0  DrDoS_DNS  
3          0.0          0.0        0.0        0.0  DrDoS_DNS  
4          0.0          0.0        0.0        0.0  DrDoS_DNS  

[5 rows x 39 columns]

In [5]:
X = df.drop(columns=[' Label'])
y = df[' Label']

In [6]:
input_features = len(X.columns)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=42, stratify=y)

___
# Select the top 20 features

In [8]:
start = time.time()
# Feature Selection
fs_start = time.time()
selector = SelectKBest(f_classif, k=20).fit(X_train, y_train)
X_train = selector.transform(X_train)
X_test = selector.transform(X_test)
fs_end = time.time()

___
# Training a model

In [9]:
# Training the model
train_start = time.time()
clf_xgb = xgb.XGBClassifier(eval_metric="logloss", seed=42)
# clf_xgb.fit(X_train, 
#             y_train,
#             # verbose=True,
#             ## the next three arguments set up early stopping.
#             early_stopping_rounds=5,
#             eval_metric='logloss',
#             eval_set=[(X_test, y_test)])
ovr = OneVsRestClassifier(clf_xgb)
#clf_xgb.fit(X_train, y_train)
ovr.fit(X_train, y_train)
train_end = time.time()
end = time.time()

c:\Users\pedro\miniconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [10]:
fs_time = (fs_end - fs_start)
train_time = (train_end - train_start)
fit_time = (end - start)

In [11]:
output_features = 20

In [12]:
plot_confusion_matrix(ovr,
                      X_test, 
                      y_test,
                      values_format='d',
                      xticks_rotation='vertical')
plt.savefig(root + 'pictures/multiclass_one_vs_rest/label_dependent/confusion_matrix_anova.pdf', bbox_inches='tight')

In [13]:
y_pred = ovr.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="macro")
recall = recall_score(y_test, y_pred, average='macro')
f1score = f1_score(y_test, y_pred, average='macro')

In [14]:
pfm = pd.DataFrame(["ANOVA", accuracy, precision, recall, f1score,
                    fit_time, fs_time, train_time, input_features, output_features])
pfm = pfm.T
pfm.columns = ["Method", "Accuracy", "Precision", "Recall", "F1_Score", 
                "Fit_Time", "FS_Time", "Train_Time", "Input_Features", "Output_Features"]

filename = root + "pickles/multiclass_one_vs_rest/hold_out/label_dependent/anova.pkl"
outfile = open(filename, 'wb')
pickle.dump(pfm, outfile)
outfile.close()

In [15]:
print(classification_report(y_test, y_pred, digits=6))

               precision    recall  f1-score   support

       BENIGN   1.000000  1.000000  1.000000        44
    DrDoS_DNS   0.620690  0.409091  0.493151        44
   DrDoS_LDAP   0.518519  0.636364  0.571429        44
  DrDoS_MSSQL   0.886364  0.886364  0.886364        44
    DrDoS_NTP   0.956522  1.000000  0.977778        44
DrDoS_NetBIOS   0.895833  0.977273  0.934783        44
   DrDoS_SNMP   0.687500  0.750000  0.717391        44
   DrDoS_SSDP   0.363636  0.363636  0.363636        44
    DrDoS_UDP   0.465116  0.454545  0.459770        44
          Syn   0.590909  0.886364  0.709091        44
         TFTP   1.000000  1.000000  1.000000        44
      UDP-lag   0.666667  0.272727  0.387097        44

     accuracy                       0.719697       528
    macro avg   0.720980  0.719697  0.708374       528
 weighted avg   0.720980  0.719697  0.708374       528

